In [1]:
%matplotlib widget
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import string

import warnings
warnings.filterwarnings("ignore")

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
#np.__version__


In [2]:
def cosh128(x):
  return 0.5*(np.exp(x,dtype=np.float128)+np.exp(-x,dtype=np.float128))

def sinh128(x):
  return 0.5*(np.exp(x,dtype=np.float128)-np.exp(-x,dtype=np.float128))
  

In [3]:
def calculate_h(X, Z, A, L, D, K, Qf, aniso, desat, num_terms):
    G = Qf/D/K
    h = np.zeros_like(X)
    A_frac = 1-desat/A
    H = D
    h = h + aniso*G*H**2/(2*L)+G*L/3+A*A_frac-2*A*A_frac/np.pi + G/(2*L)*(X**2-aniso*Z**2)-G*X
    for n in range(1,num_terms):
        npi = n*np.pi
        gamma = 2/(L*cosh128(npi*H*np.sqrt(aniso)/L))
        An = -0.5*gamma * (-4*A*A_frac*L*np.pi**2*n**3+8*G*L**2*np.pi*n**3-2*G*L**2*np.pi*n)/(npi**3*(4*n**2-1))       
        h += An * cosh128(npi*Z*np.sqrt(aniso)/L)*np.cos(npi*X/L)
    return h


In [4]:
def calc_qx(X,Z,A, L, D, K, Qf,aniso,desat,num_terms): 
    qxx = np.zeros_like(X)
    G = Qf/D/K
    A_frac = 1-desat/A
    H = D
    qxx = -K*(G/L*X-G)
    for n in range(1,num_terms):
        npi = n*np.pi
        gamma =  2/(L*cosh128(npi*H*np.sqrt(aniso)/L))
        An = -0.5*gamma * (-4*A*A_frac*L*np.pi**2*n**3+8*G*L**2*np.pi*n**3-2*G*L**2*np.pi*n)/(npi**3*(4*n**2-1))       
        qxx += -K* An * npi/L*cosh128(npi*Z*np.sqrt(aniso)/L)*(-np.sin(npi*X/L))
    return qxx

In [5]:
def calc_qz(X,Z,A, L, D, K, Qf,aniso,desat,num_terms): 
    qzz = np.zeros_like(X)
    G = Qf/D/K
    A_frac = 1-desat/A
    H = D
    Kv = K/aniso
    qzz = -Kv*aniso*(-G/L*Z)
    for n in range(1,num_terms):
        npi = n*np.pi
        gamma =  2/(L*cosh128(npi*H*np.sqrt(aniso)/L))
        An = -0.5*gamma * (-4*A*A_frac*L*np.pi**2*n**3+8*G*L**2*np.pi*n**3-2*G*L**2*np.pi*n)/(npi**3*(4*n**2-1))       
        qzz += -Kv* An * npi/L*np.sqrt(aniso)*sinh128(npi*Z*np.sqrt(aniso)/L)*np.cos(npi*X/L)
    return qzz

In [6]:
def make_tt0(A, L, D, K, vk,desat, Qf, ne ,Q_an, x0, dt=1.0,N_part=250,num_terms=100):
 dxp = x0/N_part # segments on inflow stretch, i.e. start of flow tubes
 x_part = np.linspace(dxp/2,x0-dxp/2,N_part) # place particles in middle of each flow tube
 Qzp_f = -calc_qz(x_part,np.ones(N_part)*D,A, L, D, K,Qf,vk,desat,num_terms)*dxp /Q_an # calulate inflow fraction on each flow tube 
 T_max = 36500
 nps = np.int32(T_max/dt)
 xp = np.zeros((nps,N_part))
 zp = np.zeros((nps,N_part))
 tp = np.ones((nps,N_part))*(-999)
 xp[0,:]=x_part
 zp[0,:]=D
 i = 1
 for t in np.arange(dt,T_max,dt):  
    vz = calc_qz(xp[i-1,:],zp[i-1,:],A, L, D, K,Qf,vk,desat,num_terms)/ne
    vx = calc_qx(xp[i-1,:],zp[i-1,:],A, L, D, K,Qf,vk,desat,num_terms)/ne
    xp[i,:]=xp[i-1,:]+vx*dt
    zp[i,:]=zp[i-1,:]+vz*dt
    zp[i,:]=np.where(zp[i,:]>D,np.nan,zp[i,:])
    tp[i,:]=np.where(np.isnan(zp[i,:]),np.nan,t)
    if np.sum(Qzp_f[np.isnan(tp[i,:])])>0.99:
      break        
    i += 1
 return xp,zp,tp,Qzp_f

In [7]:
def make_tt(A, L, D, K, vk,desat, Qf, ne ,Q_an, x0, dt=1.0,N_part=250,num_terms=100):
 dxp = x0/N_part # segments on inflow stretch, i.e. start of flow tubes
 x_part = np.linspace(dxp/2,x0-dxp/2,N_part) # place particles in middle of each flow tube
 Qzp_f = -calc_qz(x_part,np.ones(N_part)*D,A, L, D, K,Qf,vk,desat,num_terms)*dxp /Q_an # calulate inflow fraction on each flow tube 
 T_max = 36500
 nps = np.int32(T_max/dt)
 xp = np.zeros((nps,N_part))
 zp = np.zeros((nps,N_part))
 tp = np.ones((nps,N_part))*(-999)
 xp[0,:]=x_part
 zp[0,:]=D
 flag = np.ones(N_part)  
 i = 1 
 for t in np.arange(dt,T_max,dt):
    for j in range(N_part):
     if flag[j]==0:
        break
     vz = calc_qz(xp[i-1,j],zp[i-1,j],A, L, D, K,Qf,vk,desat,num_terms)/ne
     vx = calc_qx(xp[i-1,j],zp[i-1,j],A, L, D, K,Qf,vk,desat,num_terms)/ne
     xp[i,j]=xp[i-1,j]+vx*dt
     zp[i,j]=zp[i-1,j]+vz*dt
     if zp[i,j]>D:
        flag[j]=0
     tp[i,j]=t
    i += 1
 return xp,zp,tp,Qzp_f

In [8]:
def regression_eq(Kh, vK, Sy, D, sb, coeffs):
    """
    Calculates ln( z_ep / (1 - z_ep) ) after Eqn. (21) in Haehnel et al. 2023, Adv Wat Res
    """
    c = coeffs
    beta = np.arctan(sb)
    result = (
        c["c0"]
        + c["c1"] * np.log(np.tan(beta))
        + c["c2"] * np.log(Kh)
        + c["c3"] * np.log(D)
        + c["c4"] * np.log(Sy)
        + c["c5"] * (np.log(Kh) ** 2)
        + c["c6"] * (np.log(D) ** 2)
        + c["c7"] * np.log(np.tan(beta)) * np.log(Kh)
        + c["c8"] * np.log(np.tan(beta)) * np.log(D)
        + c["c9"] * np.log(np.tan(beta)) * np.log(Sy)
        + c["c10"] * np.log(Kh) * np.log(D)
        + c["c11"] * np.log(Kh) * np.log(Sy)
        + c["c12"] * np.log(Kh) * np.log(vK)
        + c["c13"] * np.log(D) * np.log(Sy)
        + c["c14"] * np.log(D) * np.log(vK)
    )
    return result

def z_ep(Kh, vK, Sy, D, beta, coeffs):
    """
    Calculates z_ep = exp(f) / (1 + exp(f))
    """
    logit_val = regression_eq(Kh, vK, Sy, D, beta, coeffs)
    return np.exp(logit_val) / (1 + np.exp(logit_val))

# === Coefficients ===
coeffs = {
    "c0":  1.637e-1,
    "c1": -8.430e-1,
    "c2": -4.821e-1,
    "c3": -3.063e-1,
    "c4":  4.627e-1,
    "c5": -1.427e-2,
    "c6":  4.917e-2,
    "c7": -1.180e-2,
    "c8":  4.326e-2,
    "c9":  1.659e-2,
    "c10": -1.315e-2,
    "c11":  2.875e-2,
    "c12":  2.334e-2,
    "c13":  1.857e-2,
    "c14":  3.195e-2,
}



In [9]:
def make_sens(parmsi,num_terms):
 Q_sens = np.zeros(len(parmsi))
 Qan=np.zeros(len(parmsi)+1)
 for i in range(len(parmsi)+1):
    parms=np.copy(parmsi) 
    if i < len(parms):
       parms[i]=parmsi[i]*1.01 
    A=parms[0]
    Qf=parms[1]
    D=parms[2]
    sb=parms[3]
    K=parms[4]
    vk=parms[5]
    Sy=parms[6]
    L = 2*A/sb
    val_zep = z_ep(K*0.5/A, vk, Sy, D/A, sb, coeffs)
    desat = (A-(A*val_zep))*0.75
    ncol=1000
    dx = L/ncol
    xi = np.arange(dx/2,dx*ncol+dx/2,dx)
    qz = calc_qz(xi,D,A, L, D, K,Qf,vk,desat,num_terms)
    Qan[i]=np.abs(dx*qz[qz<0].sum())

 if Qan[-1]!=0 and qz[0]<=0:
     Q_sens = np.abs((Qan[0:len(parms)]-Qan[-1])/Qan[-1])/0.01
 else:
     Q_sens=Q_sens*np.nan
     Qan[-1]=0   
 Q_sens=Q_sens/np.max(Q_sens) 
 return L, desat, Qan[-1],Q_sens






plt.ioff()
fig=plt.figure()

fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False

fig.set_size_inches((10,6.5))

A = 1.35
Qf = 0.75
D = 40                           
sb = 0.025
K = 10
vk = 2
Sy = 0.25

parms = [A,Qf,D,sb,K,vk,Sy]





num_terms=10
L, desat, Qan, sens = make_sens(parms,num_terms)

ncol = 10000
dx=L/ncol
xi = np.arange(dx/2,L+dx/2,dx)
idx = np.argmin(np.abs(calc_qz(xi,np.ones(ncol)*D,A, L, D, K,Qf,vk,desat,num_terms)))
x0 = xi[idx]
#print(x0)
N_part = 100
dt = 1.0

tp=np.array([[np.nan]])
zp=np.array([[np.nan]])
xp=np.array([[np.nan]])
    
#xp,zp,tp,Qzp_f = make_tt(A, L, D, K, vk,desat, Qf,Sy, Qan, x0, dt,N_part,num_terms)


tp[tp<-990]=np.nan
zp[zp==0]=np.nan
xp[xp==0]=np.nan
tt = np.nanmax(tp,axis=0)/365
Qzp_f = 0

plt.subplot(2,2,3)
#plt.plot(np.flipud(tt),np.cumsum(np.flipud(Qzp_f)))
plt.grid(linestyle='--',alpha=0.5)
plt.title('Residence time distribution')
plt.xlabel('Residence time (years)')
plt.ylabel('Fraction of USP flux (-)')

plt.subplot(2,1,1)
ncol=200
nlay=200
x_values = np.linspace(0, L, ncol)
y_values = np.linspace(0, D, nlay)
X, Y = np.meshgrid(x_values, y_values)
h = calculate_h(X, Y, A, L, D,K,Qf, vk, desat,num_terms)
plt.contourf(X, Y-D, h, np.linspace(0,A-desat,20), cmap='gray',alpha=0.4)
cbar = plt.colorbar()
cbar.set_label('Hydraulic head (m)')
plt.xlabel('Length of intertidal zone (m)')
plt.ylabel('Elevation a.m.s.l (m)')
res = np.nansum(tt*Qzp_f)
plt.title('Q$_{USP}$ = %1.2f m$^2$/d' % (Qan))


qz = calc_qz(X,Y,A, L, D, K,Qf,vk, desat,num_terms)
qx = calc_qx(X,Y,A, L, D, K,Qf,vk,desat,num_terms)
v = np.sqrt(qx**2+qz**2)*2
strplot=plt.streamplot(X, Y-D, qx, qz, density=0.5, linewidth=v,color='k')
plt.plot(xp,zp-D,color='r',linewidth = 0.5,alpha=1.0)

plt.ylim(-D,0)
plt.xlim(0,L)
#plt.gca().set_aspect('equal')


plt.subplot(2,2,4)
plt.bar(['A','Q$_{fresh}$','D','sb','K$_h$','vk','Sy'],sens)
plt.title('Normalized parameter sensitivies for Q$_{USP}$')

plt.tight_layout()


Amp=widgets.FloatSlider(
    value=1.35,
    min=0.05, 
    max=5.0,  
    step=0.05,
    description="Amplitude A (m)",
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'},
    continuous_update=False,
    orientation='horizontal'
)

Qfresh=widgets.FloatSlider(
    value=0.75,
    min=0.00, 
    max=10.0,  
    step=0.05,
    description="Freshwater Flux Qf (m2/d)",
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'},
    continuous_update=False,
    orientation='horizontal'
)

Depth=widgets.FloatSlider(
    value=40,
    min=5.0, 
    max=200,  
    step=5,
    description="Aquifer Depth D (m)",
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'},
    continuous_update=False,
    orientation='horizontal'
)


Slope=widgets.FloatSlider(
    value=0.025,
    min=0.01, 
    max=0.2,  
    step=0.005,
    description="Beach Slope sb (-)",
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'},
    continuous_update=False,
    orientation='horizontal',
    readout_format = '.3f'
)

HydrK=widgets.FloatLogSlider(
    value=10.0,
    base=10,
    min=0, 
    max=3,  
    step=0.01,
    description="Hydraulic Cond. Kh (m/d)",
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'},
    continuous_update=False,
    orientation='horizontal'
)

Aniso=widgets.FloatSlider(
    value=2.0,
    min=1.0, 
    max=20.0,  
    step=1.0,
    description="Vertical Anisotropy vk (-)",
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'},
    continuous_update=False,
    orientation='horizontal'
)

Yield=widgets.FloatSlider(
    value=0.25,
    min=0.05, 
    max=0.6,  
    step=0.05,
    description="Specific Yield Sy (-)",
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'},
    continuous_update=False,
    orientation='horizontal'
)

Numterms=widgets.IntSlider(
    value=10,
    min=1, 
    max=100,  
    step=1,
    description="No. of Fourier terms",
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'},
    continuous_update=False,
    orientation='horizontal'
)

Numpart=widgets.IntSlider(
    value=100,
    min=10, 
    max=1000,  
    step=10,
    description="No. of Particles",
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'},
    continuous_update=False,
    orientation='horizontal'
)

Deltat=widgets.FloatSlider(
    value=1.0,
    min=0.2, 
    max=20.0,  
    step=0.2,
    description="Time step (d)",
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'},
    continuous_update=False,
    orientation='horizontal'
)


RTD = widgets.Checkbox(
    value=False,
    description='Residence time distribution ? (needs some computation time)',
    disabled=False,
    layout=widgets.Layout(width='100%')
)



def update(change):

    A = Amp.get_interact_value()
    Qf = Qfresh.get_interact_value()
    D = Depth.get_interact_value()                           
    sb = Slope.get_interact_value()
    K = HydrK.get_interact_value()
    vk = Aniso.get_interact_value()
    Sy = Yield.get_interact_value()
    num_terms=Numterms.get_interact_value()
    N_part=Numpart.get_interact_value()
    dt=Deltat.get_interact_value()
    rtd_flag = RTD.get_interact_value()
    
    parms = [A,Qf,D,sb,K,vk,Sy]

    L, desat, Qan, sens = make_sens(parms,num_terms)    

    plt.text(-5,2,'Computing ...',fontsize=30,
           zorder=100,bbox=dict(facecolor='w', edgecolor='k'))
    fig.canvas.draw()
      
    tp=np.array([[np.nan]])
    zp=np.array([[np.nan]])
    xp=np.array([[np.nan]])
                
    if (rtd_flag and Qan>0.001):
        ncol = 10000
        dx=L/ncol
        xi = np.arange(dx/2,L+dx/2,dx)
        idx = np.argmin(np.abs(calc_qz(xi,np.ones(ncol)*D,A, L, D, K,Qf,vk,desat,num_terms)))
        x0 = xi[idx]
      #  print(x0)
       # N_part = 100
       # dt = 1.0
        xp,zp,tp,Qzp_f = make_tt(A, L, D, K, vk,desat, Qf,Sy, Qan, x0, dt,N_part,num_terms)


    tp[tp<-990]=np.nan
    zp[zp==0]=np.nan
    xp[xp==0]=np.nan
    tt = np.nanmax(tp,axis=0)/365

    fig.clf() 
    plt.subplot(2,2,3)
    if (rtd_flag and Qan>0.001):            
      plt.plot(np.flipud(tt),np.cumsum(np.flipud(Qzp_f)))
    plt.grid(linestyle='--',alpha=0.5)
    plt.title('Residence time distribution')
    plt.xlabel('Residence time (years)')
    plt.ylabel('Fraction of USP flux (-)')

    plt.subplot(2,1,1)
    ncol=200
    nlay=200
    x_values = np.linspace(0, L, ncol)
    y_values = np.linspace(0, D, nlay)
    X, Y = np.meshgrid(x_values, y_values)
    h = calculate_h(X, Y, A, L, D,K,Qf, vk, desat,num_terms)
    plt.contourf(X, Y-D, h, np.linspace(0,A-desat,20), cmap='gray',alpha=0.4)
    cbar = plt.colorbar()
    cbar.set_label('Hydraulic head (m)')
    plt.xlabel('Length of intertidal zone (m)')
    plt.ylabel('Elevation a.m.s.l (m)')
    if (not rtd_flag or Qan <= 0.001):
        plt.title('Q$_{USP}$ = %1.2f m$^2$/d' % (Qan))
    else:    
        plt.title('Q$_{USP}$ = %1.2f m$^2$/d, Flux-weighted mean residence time: %1.3f years' % (Qan,np.nansum(tt*Qzp_f)))

    
    qz = calc_qz(X,Y,A, L, D, K,Qf,vk, desat,num_terms)
    qx = calc_qx(X,Y,A, L, D, K,Qf,vk,desat,num_terms)
    v = np.sqrt(qx**2+qz**2)/((Qan+Qf)/D/Sy)
    strplot=plt.streamplot(X, Y-D, qx, qz, density=0.5, linewidth=v,color='k')
    if Qan>0.001:
      plt.plot(xp,zp-D,color='r',linewidth = 0.5,alpha=1.0)

    plt.ylim(-D,0)
    plt.xlim(0,L)
    #plt.gca().set_aspect('equal')

    plt.subplot(2,2,4)
    plt.bar(['A','Q$_{fresh}$','D','sb','K$_h$','vk','Sy'],sens)
    plt.title('Normalized parameter sensitivies for Q$_{USP}$')
    
    plt.tight_layout()
    fig.canvas.draw()
    





Amp.observe(update, 'value')
Qfresh.observe(update, 'value')
Depth.observe(update, 'value')
Slope.observe(update, 'value')
HydrK.observe(update, 'value')
Aniso.observe(update, 'value')
Yield.observe(update, 'value')
Numterms.observe(update, 'value')
Numpart.observe(update, 'value')
Deltat.observe(update, 'value')
RTD.observe(update,'value')

vbox1=widgets.VBox(
    [   Amp,
        Qfresh,
        Depth,
        Slope,
        Numterms])

vbox2=widgets.VBox([
        HydrK,
        Aniso,
        Yield,
        RTD,
        Numpart,
        Deltat])

box_layout = widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px')
 

vbox1.layout = box_layout
vbox2.layout = box_layout
 

vbox1.layout.width = '500px'


whbox=widgets.HBox([vbox1, vbox2])
widgets.VBox([fig.canvas,whbox])

<ipython-input-9-f62a7f7767dd>:83: RuntimeWarning: All-NaN slice encountered
  tt = np.nanmax(tp,axis=0)/365
